Test to chunk and make embeddings

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

c:\Users\Anthony\Documents\ragggg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
##searching if their is missing data in the csv
##missing 31 stories because riot still doesn't wrote it 
df = pd.read_csv("../data/lore.csv")
print(df.isnull().sum())
null_data = df[df.isnull().any(axis=1)]
df.fillna("L'éditeur Riot Games n'a pas encore fourni l'histoire de ce champion", inplace = True)
df
#null_data

Name          0
Region        0
SumUp         0
Biography     0
Story        31
dtype: int64


,Name,Region,SumUp,Biography,Story
0,AATROX,RUNETERRA,"Autrefois, Aatrox et ses frères étaient honoré...","Qu'on la prenne pour un dieu ou pour un démon,...",Les ténèbres. Le souffle que je ne peux pas pr...
1,AHRI,IONIA,"Connectée à la magie du royaume spirituel, Ahr...","Pendant le plus clair de sa vie, Ahri n'a rien...",Le marché embaumait l'encens et les légumes po...
2,AKALI,IONIA,Ayant abandonné l'Ordre Kinkou et le titre de ...,Ionia a toujours été une terre de magie sauvag...,« Aïe… Hé ! Bo'lii ! Tu n'y vas pas de main mo...
3,AKSHAN,SHURIMA,"Se jouant du danger, Akshan combat le mal sans...","Agissant dans l'ombre, un preux justicier traq...",Shadya n'était morte que depuis quelques semai...
4,ALISTAR,RUNETERRA,Alistar est un guerrier redoutable cherchant à...,"Bien des civilisations ont résisté à Noxus, ma...",L'éditeur Riot Games n'a pas encore fourni l'h...
...,...,...,...,...,...
166,ZERI,ZAUN,Zeri est une jeune femme vive et téméraire ori...,"Originaire d'une famille ouvrière, Zeri a gran...","« Je ne peux pas accepter », déclara le commer..."
167,ZIGGS,ZAUN,Amoureux des grosses bombes et des mèches cour...,Ziggs naquit avec un talent certain pour le br...,"Très cher Zaun. Je suis là, je suis pelucheux ..."
168,ZILEAN,RUNETERRA,"Autrefois membre du conseil d'Icathia, Zilean ...",Icathia n'a pas toujours été la plus désolée d...,L'éditeur Riot Games n'a pas encore fourni l'h...
169,ZOÉ,TARGON,"Incarnation de l'espièglerie, de l'imagination...",Conformément à la nature de la Manifestation t...,"Dès qu'elle pensa à la boutique de gâteaux, Zo..."


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

liste_chunk = []

for index, row in df.iterrows():
    text_biography = row['Biography']
    text_story = row['Story']
    texts_bio = text_splitter.create_documents([text_biography],[{"Source": "Biography", "Name": row['Name'], "Region": row['Region'], "Sum_up": row['SumUp']}])
    text_story = text_splitter.create_documents([text_story], [{"Source": "Story", "Name": row['Name'], "Region": row['Region'], "Sum_up": row['SumUp']}])
    liste_chunk.extend(texts_bio)
    liste_chunk.extend(text_story)


liste_chunk[1].__dict__


{'id': None,
 'metadata': {'Source': 'Biography',
  'Name': 'AATROX',
  'Region': 'RUNETERRA',
  'Sum_up': "Autrefois, Aatrox et ses frères étaient honorés pour avoir défendu Shurima contre le Néant. Mais ils finirent par devenir une menace plus grande encore pour Runeterra : la ruse et la sorcellerie furent employées pour les battre. Cependant, après des siècles d'emprisonnement, Aatrox fut le premier à retrouver sa liberté, en corrompant et transformant les mortels assez stupides pour tenter de s'emparer de l'arme magique qui contenait son essence. Désormais en possession d'un corps qu'il a approximativement transformé pour rappeler son ancienne forme, il arpente Runeterra en cherchant à assouvir sa vengeance apocalyptique."},
 'page_content': "Shurima fut conduit devant le Disque solaire pour devenir l'avatar d'une entité céleste aujourd'hui oubliée. Transfiguré, il obtint des ailes dorées comme la lumière de l'aube. Son armure étincelait comme une constellation d'espoir d'au-delà d

In [ ]:

liste = []
for doc in liste_chunk:
    row = {
   "page_content": doc.page_content,
   "Source": doc.metadata["Source"],
   "Name": doc.metadata["Name"],
   "Region": doc.metadata["Region"],
   "Sum_up": doc.metadata["Sum_up"]
    }
    liste.append(row)

df_lore = pd.DataFrame(liste)
df_lore
df.to_csv('../data/processed/lore_chuked.csv', sep='\t')


In [5]:
df = pd.read_csv("../data/processed/lore_chunked.csv", sep="\t")
df

,Unnamed: 0,page_content,Source,Name,Region,Sum_up
0,0,"Qu'on la prenne pour un dieu ou pour un démon,...",Biography,AATROX,RUNETERRA,"Autrefois, Aatrox et ses frères étaient honoré..."
1,1,Shurima fut conduit devant le Disque solaire p...,Biography,AATROX,RUNETERRA,"Autrefois, Aatrox et ses frères étaient honoré..."
2,2,était son nom. Il était à l'avant-garde de tou...,Biography,AATROX,RUNETERRA,"Autrefois, Aatrox et ses frères étaient honoré..."
3,3,fit appel à lui pour mater la rébellion d'Icat...,Biography,AATROX,RUNETERRA,"Autrefois, Aatrox et ses frères étaient honoré..."
4,4,toute vie devant lui. Après des années de comb...,Biography,AATROX,RUNETERRA,"Autrefois, Aatrox et ses frères étaient honoré..."
...,...,...,...,...,...,...
7384,7384,"; la douceur des fleurs d'oranger, l'arôme des...",Story,ZYRA,IXTAL,Née au cours d'une ancienne catastrophe magiqu...
7385,7385,d'une couleur qu'Hatilly n'avait jamais vue. U...,Story,ZYRA,IXTAL,Née au cours d'une ancienne catastrophe magiqu...
7386,7386,doit être plus… fertile. » Hatilly eut des vis...,Story,ZYRA,IXTAL,Née au cours d'une ancienne catastrophe magiqu...
7387,7387,"pas les vrilles la plaquer au sol, les aiguill...",Story,ZYRA,IXTAL,Née au cours d'une ancienne catastrophe magiqu...


In [13]:
from langchain.schema import Document
from langchain.vectorstores import FAISS ## used faiss cpu for my setup
from langchain.embeddings import SentenceTransformerEmbeddings

documents = [
    Document(page_content=row["page_content"], 
             metadata={
                 "Source": row.get("Source"),
                 "Name": row.get("Name"),
                 "Region": row.get("Region"),
                 "Sum_up": row.get("Sum_up")
             })
    for _, row in df.iterrows()
]

embedding_model = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)



# embeddings_model = model.encode(df["page_content"].tolist(),
#                                 batch_size=32,        
#                                 show_progress_bar=True, 
#                                 convert_to_numpy=True, )

# print(embeddings_model.shape)

vectorstore = FAISS.from_documents(documents, embedding_model)

In [20]:
results = vectorstore.similarity_search("Quelles sont les histoires d'Aatrox ?", k=5)

for r in results:
    print("=== Document ===")
    print(r.page_content)
    print(r.metadata)

=== Document ===
aux corbeaux pour sa piètre moisson et revenant d'entre les morts sous la forme d'un démon. Demacia. Ixtal. Piltover. Ionia. Shurima. Ces mythes perdurent aux quatre coins de Runeterra, modifiés, réécrits et transmis de génération en génération par de nombreux conteurs. Des histoires parlant d'une
{'Source': 'Biography', 'Name': 'FIDDLESTICKS', 'Region': 'RUNETERRA', 'Sum_up': "Quelque chose s'est éveillé à Runeterra. Quelque chose d'ancien. Quelque chose de terrible. L'horreur intemporelle connue sous le nom de Fiddlesticks arpente les frontières des civilisations mortelles, attirée dans les régions en proie à la paranoïa, où elle se repaît de ses victimes terrorisées. Armée d'une faux ébréchée, la créature décharnée moissonne la peur elle-même et fait voler en éclats l'esprit de ceux qui ont eu la malchance de lui survivre. Prenez garde aux cris des corbeaux et aux chuchotements de la silhouette qui vous paraît humaine… car Fiddlesticks est de retour."}
=== Document 

In [15]:
from rapidfuzz import process, fuzz

user_input = "Atrox"  # exemple avec typo
all_names = df["Name"].unique()

best_match = process.extractOne(user_input.upper(), all_names, scorer=fuzz.ratio)

In [ ]:
chunks_for_champion = df[df["Name"] == best_match[0]]

In [ ]:
prompt = f"""
Le contexte de la question est en dessous.
---------------------
{retrieved_chunk}
---------------------
Tu es un expert geek et connais tout l'histoire du jeu League of Legends. En t'appuyant sur le contexte la question et de la question posée, peux tu répondres de façon claire et concise.
Query: {question}
Answer:
"""